Lo primero que hacemos en el proceso de ETL de este proyecto es cargar el csv, lo cargaremos con polars usando un read_csv y lo cargaremos con el codigo ISO.


In [ ]:
import polars as pl

df = pl.read_csv("data.csv", encoding="ISO-8859-1", infer_schema_length=10000)
df



Ahora sobre el dataset vamos a analizar si hay datos que limpiar:

Primero comprobaremos si el dataset contiene valores nulos.

In [ ]:
df.null_count()


Como vemos hay muchos nulls, dado que el objetivo es agrupar por tipo de clientes el custumerID es esencial y no podemos tabajar sin el. y la descripcion seguramente la ignoremos, por que lo mas util a la hora de agrupar por cliente es la cantidad comprada, el valor total de gasto  y pais.

In [ ]:
df = df.filter(df["Description"].is_not_null())
df = df.filter(df["CustomerID"].is_not_null())
df

Habiendonos desecho de todos los datos con valores nulos seguiremos con los valores duplicados de todas las columnas, que tambien es importante por si a la hora de insertar los datos se cometio el error de añadir la misma compra dos veces, con lo cual no nos seria util para nuestra posterior predicción.

In [ ]:

f"Filas duplicadas: {df.height - df.unique().height}"

En nuestro caso elimineramos estas filas duplicadas y, como mencionamos anteriormente, es muy probable que se trate de un error de la aplicacion al insertar datos y nos perjudique a la hora de hacer las predicciones.

In [ ]:

df = df.unique()
df

Ahora iremos con los valores negativos tanto en la columna de Quantity como en el UnitPrice, ya que serian los unicos que podrian tener unos valores negativos.

In [ ]:
df.filter((pl.col("Quantity") < 0) | (pl.col("UnitPrice") < 0))


En este caso vemos que hay valores negativos en estas dos columnas, se podrian interpretar como devoluciones de los productos, pero en este caso no los vamos a usar de esta forma porque no tendria sentido contar con estos valores si solo queremos calcular las ventas y no devoluciones. Asi que los tendremos que eliminar.

In [ ]:
df = df.filter(~((pl.col("Quantity") < 0) | (pl.col("UnitPrice") < 0)))
df


Es el turno de la fecha de facturacion, vamos a transformar la columna a tipo date

In [ ]:
df = df.with_columns(pl.col("InvoiceDate").str.strptime(pl.Date, "%m/%d/%Y %H:%M"))
df


Lo siguiente sera buscar algun outliers, para evitar la distorsion de los patrones, la no normalizacion de los datos y que no afecten a la precisión del modelo.

In [ ]:
df.describe()


no vamos a liminar las compras que tengan precio de 0, porque entendemos que son regalos de la tienda y ademas nos puede venir bien para agrupar clientes que solo compren cuando hay cosas gratuitas

Siguiendo con las cosas un poco raras que habiamos detectado, en la columna Country habiamos visto un "unspecified"

In [ ]:
df.filter(pl.col("Country") == "Unspecified" )


Como en nuestro caso nos parece util y queremos llegar a agrupar las ventas por paises debemos eliminar las filas que contengan "Unspecified" en la columna Country.

In [ ]:
df = df.filter(pl.col("Country") != "Unspecified" )
df

Algo raro tambien es que habia valores maximos muy altos en la cantidad y en el precio, es decir tenemos claramente outliers que habiamos detectado antes. Para solucionar esto usaremos la formula del IQR para ddefinir un limite superior e inderior en los valores.
los modelos de agrupacion son muy sensibles a los outliers

In [ ]:
def calc_iqr_from_column_name(name:str)-> None:
    Q1 = df[name].quantile(0.25)
    Q3 = df[name].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    print(f"\nIQR for {name}")
    print(Q1, Q3, IQR, lower_bound, upper_bound)

calc_iqr_from_column_name("Quantity")
calc_iqr_from_column_name("UnitPrice")

In [ ]:
upper_bound_limit_condition = (pl.col("Quantity") > 27) | (pl.col("UnitPrice") > 7.5)

df.filter(upper_bound_limit_condition)

LLevando a cabo esta accion estariamos eliminando bastantes datos, pero la parte positiva es que estariamos normalizando lo maximo posible el flujo de los datos, entonces procederemos a eliminarlos.

In [ ]:
df = df.filter(~((pl.col("Quantity") > 27) | (pl.col("UnitPrice") > 7.5)))
df

In [ ]:
df.describe()

Viendo que el objetivo es agrupar en diferentes tipos de clientes eliminaremos las columnas que no son relevantes

CustomerID,InvoiceDate,Quantity, UnitPrice,InvoiceNo, Country: serian las columnas mas interesantes para nuestro modelo de clustering, teniendo en cuenta que country seria opcional y solo seria util si queremos segmentar internacionalmente

In [ ]:
df = df["CustomerID","InvoiceDate","Quantity", "UnitPrice","Country","InvoiceNo"]
df

Ahora es el turno de las columnas extra.
Hemos pensado las siguientes:

TotalPrice ->  Para saber cuanto gastan los clientes.
Last_buy ->  Días desde la última compra.
Frequency ->  Número de compras (facturas únicas) por cliente.
Monetary -> Suma del gasto total por cliente.

Creando la variable de totalPrice

In [ ]:
df = df.with_columns(
    (pl.col("Quantity") * pl.col("UnitPrice")).alias("TotalPrice")
)
df

ahora Last_buy, frequency y monetary

Ahora agruparemos por dias para ver la suma de todas las ventas totales. De esta forma tendremos una mejor visualizacion de todas las ventas totales en cada dia que se han realizado.


In [ ]:
max_date = df.select(pl.col("InvoiceDate").max()).to_series()[0]

df = df.group_by("CustomerID").agg([

    ((pl.lit(max_date) - pl.col("InvoiceDate").max()) / pl.duration(days=1)).alias("Last_buy"),


    pl.col("InvoiceNo").n_unique().alias("Frequency"),


    pl.col("TotalPrice").sum().alias("Monetary")
])
df


Ahora es el turno de otro describe con las nuevas variables

In [ ]:
df.describe()

como vemos en monatery y en frequency tenemos claramente outliers, aplicaremos el logaritmo para no perder informacion y reducir la gran diferencia que hay, eso si tenemos que tener cuidado con los ceros

no lo aplicaremos sobre last_buy porque no tiene una destribucion tan extrema como monetary o frequency y los dias son mas uniformes lo que puede probocar que el logaritmo distorsione la interpretacion.

In [ ]:
df = df.filter(df["Monetary"]>0)
df = df.with_columns(
    (pl.col("Monetary") + 1).log().alias("Monetary_log"),
    (pl.col("Frequency") + 1).log().alias("Frequency_log")
)
df

Con esto tendriamos casi listo el proceso de ETL

Faltaria hacer algunos pasos que son recomendados como el escalado dde variables ya que las metricas estan en escalas muy diferentes.

y una gran diferencia en a escala afecta al clustering
usaremos un standarscaler, tambien prodiamos usar el minmaxscaler o el Z

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.select(["Last_buy", "Frequency_log", "Monetary_log"]).to_numpy()


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
df = df.with_columns([
    pl.Series("Last_buy_scaled", X_scaled[:, 0]),
    pl.Series("Frequency_scaled", X_scaled[:, 1]),
    pl.Series("Monetary_scaled", X_scaled[:, 2])
])
df

Aqui hacemos un ultimo describe para ver como ha quedado el dataset, con todos los cambios

In [ ]:
df.describe()

In [ ]:
df.write_csv("data_cleaned(clustering).csv")




In [ ]:
import pickle


with open("datos_grouped(clustering).pkl", "wb") as file:
    pickle.dump(df, file)

